# Introduction

Le **traduction automatique**, tout comme d'autres tâches de NLP,  a connu des évolutions majeures ces dernières années grâce au _Deep Learning_.

Ce notebook présente un exemple simple de traduction, utilisant des **réseaux de neurones récurrents (RNN)**.

Le but est d'entraîner un modèle à **traduire différents formats de dates vers un format unique**.

# Imports

In [125]:
import pandas as pd
import numpy as np

from string import ascii_lowercase, digits
from typing import List, Dict
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed

# Données

Pour cet exercice, on a généré au préalable un dataset de 25 000 dates différentes entre le `01/01/1800` et le `31/12/2050`.

On a réparti ces données en deux datasets:
- `train_data`: 10 000 pour l'entraînement du modèle
- `test_data`: 15 000 pour l'évaluation

Chaque dataset a deux colonnes:
- `input`: la date dans l'un des différents formats sources
- `target`: la date au format cible

Commençons par charger les données et un aperçu de celles-ci:

In [97]:
train_data = pd.read_csv("./data/train_dates.csv", sep=";")
test_data = pd.read_csv("./data/test_dates.csv", sep=";")

train_data.head()

,input,target
0,Sunday 17 Jun 1866,17-06-1866
1,"Thursday, 28 Aug. 1958",28-08-1958
2,"Thursday, 1985/01/17",17-01-1985
3,1937 03 24,24-03-1937
4,"Saturday, 1831 April 09",09-04-1831


On vérifie ensuite la taille des datasets, et on s'assure qu'il n'y a aucune date en commun entre `train_data` et `test_data`:

In [98]:
print("Taille du dataset d'entraînement:", len(train_data))
print("Taille du dataset d'évaluation:", len(test_data))
print("Nombre de dates communes entre `train_data` et `test_data`:", 
      len(set(train_data['target'].values).intersection(set(test_data['target'].values))))

Taille du dataset d'entraînement: 10000
Taille du dataset d'évaluation: 15000
Nombre de dates communes entre `train_data` et `test_data`: 0


# Feature engineering

Les données qu'on manipule sont des **données textuelles**. 

Afin de pouvoir les traiter avec un modèle de Machine Learning, il faut d'abord réussir à les **transformer en _features_ (caractéristiques) numériques**. C'est l'objectif de cette section.

## Tokenisation

Un texte d'entrée est d'abord **_tokenisé_**, c'est-à-dire découpé en une **séquence d'unités de texte (_tokens_)**. 

Pour cet exemple simple, nos unités de texte seront **les caractères**. La traduction consistera donc à prédire une séquence de caractères à partir d'une autre séquence de caractères.

Il est à noter que dans les vrais modèles de traduction, les _tokens_ sont plutôt des mots, ou des parties de mots (préfixes, racines, suffixes...).

In [119]:
def tokenize(date_str: str) -> str:
    return [c for c in date_str]

In [113]:
example_date = "Thursday, 28 Aug. 1958"
tokenized_date = tokenize(example_date)

print("Exemple de date:")
print(example_date)
print()
print("Résultat de la tokenisation:")
print(tokenized_date)

Exemple de date:
Thursday, 28 Aug. 1958

Résultat de la tokenisation:
['T', 'h', 'u', 'r', 's', 'd', 'a', 'y', ',', ' ', '2', '8', ' ', 'A', 'u', 'g', '.', ' ', '1', '9', '5', '8']


## Vocabulaire

L'ensemble des _tokens_ distincts dans la langue d'entrée constitue le **vocabulaire**.

On a choisi de faire une **_tokenisation_ niveau caractères** afin d'avoir un vocabulaire de taille réduite. Pour le réduire encore plus, on va remplacer tous les caractères spéciaux autres que le tiret `-` par des espaces ` `, et toutes les majuscules par des minuscules. Un vocabulaire réduit simplifie la tâche de traduction, et peut donc potentiellement accélérer la convergence du modèle.

- Le vocabulaire d'entrée se limite alors aux caractères alphanumériques minuscules, en plus de l'espace ` ` et du tiret `-`.
- Le vocabulaire cible est encore plus réduit, il est constitué des chiffres de `0` à `9` et du tiret `-`.

In [120]:
input_vocabulary = list(digits) + list(ascii_lowercase) + [' ', '-']
target_vocabulary = list(digits) + ['-']

print("Vocabulaire d'entrée:")
print(input_vocabulary)
print()
print("Vocabulaire cible:")
print(target_vocabulary)

Vocabulaire d'entrée:
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '-']

Vocabulaire cible:
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-']


## Encodage

Un encodage possible du texte en _features_ numériques est de remplacer chaque caractère par son index dans le vocabulaire. 

Pour simplifier l'encodage, on va construire un mapping `(caractère > index)` pour le vocabulaire d'entrée, et de même pour le vocabulaire cible.

Pour simplifier le décodage des prédictions du modèle (qui vont être numériques), on va également construire un mapping `(index > caractère)` pour le vocabulaire cible.

In [123]:
input_vocab_char_to_idx = {char: idx for (idx, char) in enumerate(input_vocabulary)}
target_vocab_char_to_idx = {char: idx for (idx, char) in enumerate(target_vocabulary)}
target_vocab_idx_to_char = {idx: char for (idx, char) in enumerate(target_vocabulary)}

print("Mapping (caractère > index) pour le vocabulaire d'entrée:")
print(input_vocab_char_to_idx)
print()
print("Mapping (caractère > index) pour le vocabulaire cible:")
print(target_vocab_char_to_idx)
print()
print("Mapping (index > caractère) pour le vocabulaire cible:")
print(target_vocab_idx_to_char)

Mapping (caractère > index) pour le vocabulaire d'entrée:
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'q': 26, 'r': 27, 's': 28, 't': 29, 'u': 30, 'v': 31, 'w': 32, 'x': 33, 'y': 34, 'z': 35, ' ': 36, '-': 37}

Mapping (caractère > index) pour le vocabulaire cible:
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '-': 10}

Mapping (index > caractère) pour le vocabulaire cible:
{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '-'}


Implémentons maintenant les fonctions d'encodage et de décodage pour une séquence entière. La fonction d'encodage inclut la tokenisation comme première étape.

In [140]:
def encode(date_str: str, char_to_idx_mapping: Dict[str, int]) -> List[int]:
    tokenized_date = tokenize(date_str)
    encoding = []
    for c in tokenized_date:
        c = c.lower()
        c = c if c in char_to_idx_mapping else ' '
        encoding.append(char_to_idx_mapping[c])
    return encoding

def decode(encoded_text: List[int], idx_to_char_mapping: Dict[int, str]) -> str:
    return ''.join([idx_to_char_mapping[idx] for idx in encoded_text])

In [142]:
encoded_date = encode(tokenized_date, input_vocab_char_to_idx)
example_predicted_date = [2, 6, 10, 1, 1, 10, 2, 0, 2, 1]
decoded_date = decode(example_predicted_date, target_vocab_idx_to_char)

print("Exemple de date d'entrée:")
print(example_date)
print()
print("Résultat de l'encodage:")
print(encoded_date)
print()
print("Exemple de date prédite au format encodé:")
print(example_predicted_date)
print()
print("Résultat du décodage:")
print(decoded_date)

Exemple de date d'entrée:
Thursday, 28 Aug. 1958

Résultat de l'encodage:
[29, 17, 30, 27, 28, 13, 10, 34, 36, 36, 2, 8, 36, 10, 30, 16, 36, 36, 1, 9, 5, 8]

Exemple de date prédite au format encodé:
[2, 6, 10, 1, 1, 10, 2, 0, 2, 1]

Résultat du décodage:
26-11-2021


In [153]:
def one_hot(encoded_date: List[int], vocab_size: int) -> np.ndarray:
    one_hot_encoding = np.zeros((len(encoded_date), vocab_size))
    for i, idx in enumerate(encoded_date):
        one_hot_encoding[i, idx] = 1
    return one_hot_encoding

In [154]:
one_hot([2, 6, 10, 1, 1, 10, 2, 0, 2, 1], 11)

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [27]:
def to_numeric_representation(date_str: str, vocabulary: dict, verbose: bool = False) -> np.ndarray:
    date_lower = date_str.lower()
    tokenized_date = [c for c in date_lower]
    tokenized_date_withtout_special_chars = [c if c in vocabulary else ' ' for c in tokenized_date]
    numeric_tokens = [vocabulary[c] for c in tokenized_date_withtout_special_chars]
    one_hot_encoding = np.array([one_hot(c, len(vocabulary)) for c in numeric_tokens])
    if verbose:
        print("Texte initial:\n", date_str, '\n')
        print("Texte sans majuscules:\n", date_lower, '\n')
        print("Tokens (caractères):\n", tokenized_date, '\n')
        print("Tokens sans caractères spéciaux:\n", tokenized_date_withtout_special_chars, '\n')
        print("Tokens remplacés par leurs indexes:\n", numeric_tokens, '\n')
        print("One-hot encoding:\n", one_hot_encoding)
    return one_hot_encoding

In [38]:
example = to_numeric_representation(date_str='Thursday, 1985/01/17', vocabulary=input_vocabulary, verbose=True)

Texte initial:
 Thursday, 1985/01/17 

Texte sans majuscules:
 thursday, 1985/01/17 

Tokens (caractères):
 ['t', 'h', 'u', 'r', 's', 'd', 'a', 'y', ',', ' ', '1', '9', '8', '5', '/', '0', '1', '/', '1', '7'] 

Tokens sans caractères spéciaux:
 ['t', 'h', 'u', 'r', 's', 'd', 'a', 'y', ' ', ' ', '1', '9', '8', '5', ' ', '0', '1', ' ', '1', '7'] 

Tokens remplacés par leurs indexes:
 [29, 17, 30, 27, 28, 13, 10, 34, 36, 36, 1, 9, 8, 5, 36, 0, 1, 36, 1, 7] 

One-hot encoding:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [39]:
example.shape

(20, 38)

In [40]:
def pad(one_hot_encoded: np.ndarray, max_len: int) -> np.ndarray:
    sentence_len = one_hot_encoded.shape[0]
    vocab_size = one_hot_encoded.shape[1]
    to_add = max_len - sentence_len
    return np.vstack([
        np.zeros((to_add, vocab_size)),
        one_hot_encoded
    ])

In [41]:
print("Longueur maximale des exemples d'entraînement:", train_data['input'].str.len().max())
print("Longueur maximale des exemples d'évaluation:", test_data['input'].str.len().max())

Longueur maximale des exemples d'entraînement: 28
Longueur maximale des exemples d'évaluation: 28


In [42]:
padded_example = pad(example, 28)
padded_example.shape

(28, 38)

In [43]:
to_numeric_representation("17-06-1866", target_vocabulary)

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [57]:
X_train = np.vstack(train_data['input'].map(
    lambda d: np.expand_dims(pad(to_numeric_representation(d, input_vocabulary), 28), 0)
))
y_train = np.vstack(train_data['target'].map(
    lambda d: np.expand_dims(pad(to_numeric_representation(d, target_vocabulary), 10), 0)
))

In [58]:
X_test = np.vstack(test_data['input'].map(
    lambda d: np.expand_dims(pad(to_numeric_representation(d, input_vocabulary), 28), 0)
))
y_test = np.vstack(test_data['target'].map(
    lambda d: np.expand_dims(pad(to_numeric_representation(d, target_vocabulary), 10), 0)
))

In [59]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(10000, 28, 38)
(10000, 10, 11)
(15000, 28, 38)
(15000, 10, 11)


# Modèle

In [67]:
model = Sequential()
model.add(LSTM(64, input_shape=(28, 38)))
model.add(RepeatVector(10))
model.add(LSTM(32, return_sequences=True))
model.add(TimeDistributed(Dense(11, activation='softmax')))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [68]:
model.summary(line_length=100)

Model: "sequential_2"
____________________________________________________________________________________________________
 Layer (type)                                Output Shape                            Param #        
 lstm_4 (LSTM)                               (None, 64)                              26368          
                                                                                                    
 repeat_vector_2 (RepeatVector)              (None, 10, 64)                          0              
                                                                                                    
 lstm_5 (LSTM)                               (None, 10, 32)                          12416          
                                                                                                    
 time_distributed_2 (TimeDistributed)        (None, 10, 11)                          363            
                                                                     

In [69]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.4)

Epoch 1/50
188/188 [==============================] - 5s 15ms/step - loss: 1.9976 - accuracy: 0.3031 - val_loss: 1.7676 - val_accuracy: 0.3929
Epoch 2/50
188/188 [==============================] - 2s 12ms/step - loss: 1.6354 - accuracy: 0.4554 - val_loss: 1.4699 - val_accuracy: 0.5629
Epoch 3/50
188/188 [==============================] - 2s 12ms/step - loss: 1.2856 - accuracy: 0.5996 - val_loss: 1.1391 - val_accuracy: 0.6239
Epoch 4/50
188/188 [==============================] - 2s 12ms/step - loss: 1.0316 - accuracy: 0.6429 - val_loss: 0.9454 - val_accuracy: 0.6611
Epoch 5/50
188/188 [==============================] - 2s 13ms/step - loss: 0.9025 - accuracy: 0.6713 - val_loss: 0.8803 - val_accuracy: 0.6735
Epoch 6/50
188/188 [==============================] - 2s 12ms/step - loss: 0.8286 - accuracy: 0.6887 - val_loss: 0.8039 - val_accuracy: 0.6922
Epoch 7/50
188/188 [==============================] - 2s 12ms/step - loss: 0.7777 - accuracy: 0.7026 - val_loss: 0.7596 - val_accuracy: 0.7073

In [70]:
model.evaluate(X_test, y_test)

469/469 [==============================] - 2s 3ms/step - loss: 0.0154 - accuracy: 0.9973


[0.015441731549799442, 0.9972599744796753]

In [85]:
TEST_DATE = "Ven. 26 nov. 2021"

prediction = model.predict(
    np.expand_dims(pad(to_numeric_representation(TEST_DATE, input_vocabulary), 28), 0)
)

print(''.join([reverse_target_vocabulary[idx] for idx in prediction.argmax(axis=-1)[0]]))

26-11-2021
